# 日本語Word2Vecサンプル

## Step1 青空文庫より「三四郎」をダウンロードし整形するまで

In [1]:
# ファイルダウンロード、解凍
!wget http://www.aozora.gr.jp/cards/000148/files/794_ruby_4237.zip
!unzip 794_ruby_4237.zip
!ls -l sanshiro.txt

--2018-07-13 10:37:05--  http://www.aozora.gr.jp/cards/000148/files/794_ruby_4237.zip
Resolving www.aozora.gr.jp (www.aozora.gr.jp)... 59.106.13.115
Connecting to www.aozora.gr.jp (www.aozora.gr.jp)|59.106.13.115|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.aozora.gr.jp/cards/000148/files/794_ruby_4237.zip [following]
--2018-07-13 10:37:07--  https://www.aozora.gr.jp/cards/000148/files/794_ruby_4237.zip
Connecting to www.aozora.gr.jp (www.aozora.gr.jp)|59.106.13.115|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 158711 (155K) [application/zip]
Saving to: ‘794_ruby_4237.zip’

100%[======================================>] 158,711      240KB/s   in 0.6s   

2018-07-13 10:37:09 (240 KB/s) - ‘794_ruby_4237.zip’ saved [158711/158711]

Archive:  794_ruby_4237.zip
  inflating: sanshiro.txt            
-rw-r----- 1 dsxuser dsxuser 363871 Jun 19  2014 sanshiro.txt


In [3]:
import codecs
# ファイル読込み、内部表現化
f = codecs.open('sanshiro.txt', "r", "sjis")
text = f.read()
f.close()

# ファイル整形
import re
# ヘッダ部分の除去
text = re.split('\-{5,}',text)[2]
# フッタ部分の除去
text = re.split('底本：',text)[0]
# | の除去
text = text.replace('|', '')
# ルビの削除
text = re.sub('《.+?》', '', text)
# 入力注の削除
text = re.sub('［＃.+?］', '',text)
# 空行の削除
text = re.sub('\n\n', '\n', text) 
text = re.sub('\r', '', text)

# 整形結果確認

# 頭の100文字の表示 
print(text[:100])
# 見やすくするため、空行 
print()
print()
# 後ろの100文字の表示 
print(text[-100:])



一

　うとうととして目がさめると女はいつのまにか、隣のじいさんと話を始めている。このじいさんはたしかに前の前の駅から乗ったいなか者である。発車まぎわに頓狂な声を出して駆け込んで来て、いきなり肌を


取りかかる。与次郎だけが三四郎のそばへ来た。
「どうだ森の女は」
「森の女という題が悪い」
「じゃ、なんとすればよいんだ」
　三四郎はなんとも答えなかった。ただ口の中で迷羊、迷羊と繰り返した。






## Step2 Janomeを使い三四郎テキストから単語リストを生成

In [4]:
# Janomeのインストール
!pip install janome

# Janomeのロード
from janome.tokenizer import Tokenizer

# Tokenneizerインスタンスの生成 
t = Tokenizer()

# テキストを引数として、形態素解析の結果、名詞・動詞原型のみを配列で抽出する関数を定義 
def extract_words(text):
    tokens = t.tokenize(text)
    return [token.base_form for token in tokens 
        if token.part_of_speech.split(',')[0] in['名詞', '動詞']]

#  関数テスト
ret = extract_words('三四郎は京都でちょっと用があって降りたついでに。')
for word in ret:
    print(word)

# 全体のテキストを句点('。')で区切った配列にする。 
sentences = text.split('。')
# それぞれの文章を単語リストに変換(処理に数分かかります)
word_list = [extract_words(sentence) for sentence in sentences]

# 結果の一部を確認 
for word in word_list[0]:
    print(word)

    100% |████████████████████████████████| 20.0MB 47kB/s  eta 0:00:01
  Running setup.py bdist_wheel for janome ... done
  Stored in directory: /home/dsxuser/.cache/pip/wheels/53/60/be/fe884e2d0ebc9fec0988736cf08a2820ab34e3569fc0c5a25a
Successfully built janome
三四郎
京都
ちょっと
用
ある
降りる
ついで
一
する
目
さめる
女
隣
じいさん
話
始める
いる


## Step 3 準備したデータを用いてWord2Vecのモデル作成、学習実施

In [5]:
# Word2Vecライブラリの導入
!pip install gensim  

# Word2Vecライブラリのロード
from gensim.models import word2vec

# size: 圧縮次元数
# min_count: 出現頻度の低いものをカットする
# window: 前後の単語を拾う際の窓の広さを決める
# iter: 機械学習の繰り返し回数(デフォルト:5)十分学習できていないときにこの値を調整する
# model.wv.most_similarの結果が1に近いものばかりで、model.dict['wv']のベクトル値が小さい値ばかりの 
# ときは、学習回数が少ないと考えられます。
# その場合、iterの値を大きくして、再度学習を行います。

# 事前準備したword_listを使ってWord2Vecの学習実施
model = word2vec.Word2Vec(word_list, size=100,min_count=5,window=5,iter=100)

    100% |████████████████████████████████| 23.5MB 42kB/s  eta 0:00:01
Requirement not upgraded as not directly required: numpy>=1.11.3 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from gensim)
Requirement not upgraded as not directly required: six>=1.5.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from gensim)
Requirement not upgraded as not directly required: scipy>=0.18.1 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from gensim)
Requirement not upgraded as not directly required: boto>=2.32 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from smart-open>=1.2.1->gensim)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from smart-open>=1.2.1->gensim)
Requirement not upgraded as not directly required: boto3 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from smart-open>=1.2.1->gensim)
Requirement not upgraded as not directly required: charde

In [7]:
# 結果の確認1
# 一つ一つの単語は100次元のベクトルになっています。 
# 「世間」のベクトル値を確認します。
print(model.__dict__['wv']['世間'])

[-0.1185699  -0.55271137  0.28981113 -0.77834398 -0.11299677  0.18112819
  0.56930435 -0.14783838 -0.49145272 -0.32555279  0.7454865   0.39812958
 -0.17299417 -0.78942966  0.67454261  0.12464167 -0.62386167 -0.6949895
 -0.43427449 -0.30724698  0.47436664  0.04066832 -0.52796072  0.49218211
  0.42828065  0.19826999  0.25669956 -0.30152366 -1.05436087  0.32642192
 -0.14976506 -0.34333041 -0.17113055 -0.7798264   0.57628167  0.42036498
 -0.02534406 -0.94544441  0.0987906   0.10166086 -0.25996104 -0.48275819
  0.04170883 -0.11712918  0.05164267  0.15540655  0.30475974  0.12529774
 -0.72473979  0.0481841   0.39582559  0.26715094  0.09145357 -0.16046812
 -0.36561432 -0.40758082  0.6085574  -0.28804165  0.38687402 -0.18952528
 -0.50897199 -0.01298458  0.61324769  0.52890402  0.34517321  0.06064553
  0.31997317  0.57552683  0.00670109  0.38636935  0.70543224  0.27081084
 -0.22624452  0.94744062 -0.03035834  0.13171527  0.03211259 -0.08502633
 -0.06518854  0.42222449 -0.17157423  0.3754316  -0.

In [8]:
# 結果の確認2
# 関数most_similarを使って「世間」の類似単語を調べます 
ret = model.wv.most_similar(positive=['世間']) 
for item in ret:
    print(item[0], item[1])

聞こえる 0.566158652305603
喝采 0.5400208234786987
自己 0.5261504054069519
社会 0.5239230394363403
決心 0.5216581225395203
堪える 0.5151171088218689
外国 0.4964694082736969
賛成 0.486000657081604
圧迫 0.48121926188468933
酔う 0.46692538261413574
